In [3]:
import enchant
import pandas as pd
import string
import re
from nltk.tokenize import RegexpTokenizer

In [4]:
def check_special(s):
   return all(i in string.punctuation for i in s)

In [ ]:
dic = enchant.Dict("en_US")
from transformers import MarianTokenizer, MarianMTModel

src = "en"  # source language
trg = "hi"  # target language

model_name = f"Helsinki-NLP/opus-mt-{src}-{trg}"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

In [ ]:
df = pd.read_csv('train_hinglish_english.csv')
df.head() 

In [ ]:
sample = df['Hinglish'][2]
sample

In [8]:
def preprocess_text(text):
    # utterances = text.split("##")
    # without_speaker = []
    # for utterance in utterances:
    #     if(len(utterance.strip())==0):
    #         continue
    #     utr_split = utterance.split(":")
    #     without_speaker.append(utr_split[1])
    # result = " ".join(without_speaker)
    result = text.lower()
    cleanr = re.compile('<.*?>')
    result = re.sub(cleanr, '', result)
    result=re.sub(r'http\S+', '',result)
    result = re.sub('[0-9]+', '', result)
    tokenizer = RegexpTokenizer(r'\w+')
    result = tokenizer.tokenize(result)
    result = " ".join(result)
    return result

In [ ]:
preprocess_text(sample)

In [10]:
# Refer https://github.com/precog-iiitd/hindi-english-code-mixing-lidf-ner for setting this up
os.environ['HINGLISH_ROOT_DIR'] = '/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner'

In [11]:
def convert_and_append(temp_list):
    roman_hin_inp = ' '.join(temp_list)
    batch = tokenizer([roman_hin_inp], return_tensors="pt")
    generated_ids = model.generate(**batch)
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [12]:
import ast
import subprocess
import os

def get_transliterated(text):
    # Tokenised text
    # text = [['mee'], ['too'], ['but'], ['aaj'], ['jaldi'], ['chalke'], ['dekhte'], ['hai'], [':P']]

    devanagariChars   = ['\u0900', '\u0901', '\u0902', '\u0903', '\u0904', '\u0905', '\u0906', '\u0907', '\u0908', '\u0909', '\u090a', '\u090b', '\u090c', '\u090d', '\u090e', '\u090f', '\u0910', '\u0911', '\u0912', '\u0913', '\u0914', '\u0915', '\u0916', '\u0917', '\u0918', '\u0919', '\u091a', '\u091b', '\u091c', '\u091d', '\u091e', '\u091f', '\u0920', '\u0921', '\u0922', '\u0923', '\u0924', '\u0925', '\u0926', '\u0927', '\u0928', '\u0929', '\u092a', '\u092b', '\u092c', '\u092d', '\u092e', '\u092f', '\u0930', '\u0931', '\u0932', '\u0933', '\u0934', '\u0935', '\u0936', '\u0937', '\u0938', '\u0939', '\u093a', '\u093b', '\u093c', '\u093d', '\u093e', '\u093f', '\u0940', '\u0941', '\u0942', '\u0943', '\u0944', '\u0945', '\u0946', '\u0947', '\u0948', '\u0949', '\u094a', '\u094b', '\u094c', '\u094d', '\u094e', '\u094f', '\u0950', '\u0951', '\u0952', '\u0953', '\u0954', '\u0955', '\u0956', '\u0957', '\u0958', '\u0959', '\u095a', '\u095b', '\u095c', '\u095d', '\u095e', '\u095f', '\u0960', '\u0961', '\u0962', '\u0963', '\u0964', '\u0965', '\u0966', '\u0967', '\u0968', '\u0969', '\u096a', '\u096b', '\u096c', '\u096d', '\u096e', '\u096f', '\u0970', '\u0971', '\u0972', '\u0973', '\u0974', '\u0975', '\u0976', '\u0977', '\u0978', '\u0979', '\u097a', '\u097b', '\u097c', '\u097d', '\u097e', '\u097f']
    englishCharacters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    with open('/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/transliterationModel/englishMap', 'r') as fp:
        EnglishMap = fp.read()
        EnglishMap = ast.literal_eval(EnglishMap)

        revEnglishMap = [0 for i in range(len(EnglishMap))]
        for key in EnglishMap:
            revEnglishMap[EnglishMap[key]] = key
            
    with open('/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/transliterationModel/hindiMap', 'r') as fp:
        HindiMap = fp.read()
        HindiMap = ast.literal_eval(HindiMap)

        revHindiMap = [0 for i in range(len(HindiMap))]
        for key in HindiMap:
            revHindiMap[HindiMap[key]] = key

    # print('Encoding and writing to file')

    for i in range(len(text)):

        cleanText = ''.join([ch for ch in text[i][0].lower() if ch in EnglishMap])
        englishEncoding = [str(EnglishMap[ch]) for ch in cleanText]
        text[i] += [cleanText, englishEncoding] 

    fp = open('/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/wordsToTransliterate.txt', 'w')
    for token in text:
        englishEncoding = token[2]
        if len(englishEncoding) != 0:
            fp.write(' '.join(englishEncoding) + '\n')
    fp.close()

    # print('Running transliteration script')

    ##################################################################
    #
    #	Run Transliteration model (bash script pred.sh)
    #
    process = subprocess.Popen('bash /home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/pred.sh ' + 
    '/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/transliterationModel/ ' +
    '/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/wordsToTransliterate.txt ' +
    '/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/wordsTransliterated.txt ' + 
    '/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/transliterationModel/model.ckpt-66053', shell=True, stdout=subprocess.PIPE)
    process.wait()
    #
    ##################################################################

    # print('Finished running transliteration script')

    with open('/home/ubuntu/teach-trans/hindi-english-code-mixing-lidf-ner/transliteration/wordsTransliterated.txt', 'r') as fp:
        transliteratedEncoding = fp.readlines()

    transliteratedText  = []

    for i in range(len(transliteratedEncoding)):
        transliteratedEncoding[i] = transliteratedEncoding[i].strip('\n').split(' ')
        string = ''
        for ch in transliteratedEncoding[i]:
            if ch != '':
                string += devanagariChars[int(ch)]
        transliteratedText.append(string)
        
    mark = 0

    for i in range(len(text)):
        englishEncoding = text[i][2]
        if len(englishEncoding) != 0:
            text[i] += [transliteratedEncoding[mark], transliteratedText[mark]]
            mark += 1
        else:
            text[i] += [[], ""]

    # print('Original       :', ' '.join([x[0] for x in text]))
    # print('Cleaned        :', ' '.join([x[1] for x in text]))
    # print('Transliterated :', ' '.join([x[4] for x in text]))
    return ' '.join([x[4] for x in text])

In [ ]:
sample

In [16]:
def convert_to_devanagri_hindi(input):
    li = preprocess_text(input).split()
    ans = []
    english_list = []
    roman_list = []
    for word in li:
        if check_special(word): # Special    
            if len(english_list) != 0:
                if len(english_list) < 2:                
                    english_list = [[temp] for temp in english_list]
                    transliterated = get_transliterated(english_list)
                    ans += transliterated.split()
                    english_list = []      
                else:
                # if True:
                    dev_hin = convert_and_append(english_list)                    
                    ans += dev_hin.split()
                english_list = []
            elif len(roman_list) != 0:
                roman_list = [[temp] for temp in roman_list]
                transliterated = get_transliterated(roman_list)
                ans += transliterated.split()
                roman_list = []    
            ans.append(word)    

        elif dic.check(word): # English
            if len(roman_list) != 0:
                roman_list = [[temp] for temp in roman_list]
                transliterated = get_transliterated(roman_list)
                ans += transliterated.split()
                roman_list = []      
            english_list.append(word)    

        else: # Roman Hindi
            if len(english_list) != 0:
                if len(english_list) < 2:
                    english_list = [[temp] for temp in english_list]
                    transliterated = get_transliterated(english_list)
                    ans += transliterated.split()
                    english_list = []
                else:
                # if True:
                    dev_hin = convert_and_append(english_list)
                    ans += dev_hin.split()                    
                english_list = []
            roman_list.append(word)
        

    if len(english_list) != 0:
        if len(english_list) < 2:                
            english_list = [[temp] for temp in english_list]
            transliterated = get_transliterated(english_list)
            ans += transliterated.split()
            english_list = []      
        else:
        # if True:
            dev_hin = convert_and_append(english_list)            
            ans += dev_hin.split()
        english_list = []
    elif len(roman_list) != 0:
        roman_list = [[temp] for temp in roman_list]
        transliterated = get_transliterated(roman_list)
        ans += transliterated.split()
        roman_list = []

    return ' '.join(ans)

In [ ]:
convert_to_devanagri_hindi(sample)

In [18]:
src = "hi"  # source language
trg = "en"  # target language

model_name = f"Helsinki-NLP/opus-mt-{src}-{trg}"
hi_en_tokenizer = MarianTokenizer.from_pretrained(model_name)
hi_en_model = MarianMTModel.from_pretrained(model_name)

def devanagri_hindi_to_english(input):
    batch = hi_en_tokenizer([input], return_tensors="pt")
    generated_ids = hi_en_model.generate(**batch)
    return hi_en_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
devanagri_hindi_to_english(convert_to_devanagri_hindi(sample))

In [ ]:
from sacrebleu.metrics import BLEU

refs = [[df['English'][2]]]

hyps = [devanagri_hindi_to_english(convert_to_devanagri_hindi(sample))]

bleu = BLEU(lowercase=True)

result = bleu.corpus_score(hyps, refs)
print(result)

In [22]:
from tqdm import tqdm

def convert_df_hinglish_to_english(inp_df, csv_file_name = 'output.csv'):
    devanagri_hindi_items = []
    translated_english_items = []
    
    head = inp_df.head(100)
    for index, row in tqdm(head.iterrows()):
        hinglish = row['Hinglish']
        devanagri_hindi = convert_to_devanagri_hindi(hinglish)
        translated_english = devanagri_hindi_to_english(devanagri_hindi)
        devanagri_hindi_items.append(devanagri_hindi)
        translated_english_items.append(translated_english)
    
    head['Devanagri Hindi'] = devanagri_hindi_items
    head['Translated English'] = translated_english_items

    head.to_csv(csv_file_name)
    return head

In [ ]:
convert_df_hinglish_to_english(df, 'output1.csv')

In [ ]:
def calculate_bleu(csv_file_name = 'output1.csv'):
    inp_df = pd.read_csv(csv_file_name)
    translated_english = inp_df['Translated English']
    target_english = inp_df['English']
    bleu = BLEU(lowercase=True)

    result = bleu.corpus_score(list(translated_english), [list(target_english)])
    return result
    
calculate_bleu()